# Calculation of RSE and quantitative determination of PFAS concentrations from the LC-TQ V_2
Author: Valerie de Rijk

Date: 15/8/2025

## 1 Data Loading
First we load in the raw data as a csv file (which is the file you directly get from MassHunter).
We also directly check if the s/n ratio is above 3 for all samples in the calibration line. 

In [77]:
%run src/functions_quantitative.ipynb
rawdata = load_and_label_pfas_csv("rawdata/TestSoil_20250911.csv")
sn_results = check_calibration_sn_ratios(rawdata)
# Print formatted results
print_sn_check_results(sn_results)

Detected delimiter: ','
['PFBA', '13C4-PFBA', 'PFPeA', '13C5-PFPeA', 'HFPO-DA', '13C3-HFPO-DA', 'PFBS', '13C3-PFBS', 'PFHxA', '13C5-PFHxA', '4:2 FTS', '13C2-4:2FTS', 'PFPeS', 'PFPHpA', '13C4-PFHpA', 'ADONA', 'PFHxS', '13C3-PFHxS', 'PFOA', '13C8-PFOA', '6:2 FTS', '13C2-6:2FTS', 'PFHpS', 'PFNA', '13C9-PFNA', 'PFOSA', 'PFOS', '13C8-PFOSA', '13C8-PFOS', 'NMeFOSA', 'PFDA', 'D3-NMeFOSA', '13C6-PFDA', 'NEtFOSA', '8:2 FTS', '13C2-8:2FTS', '9Cl-PF3ONS', 'D5-NEtFOSA', 'PFNS', 'PFUnA', 'NMeFOSAA', '13C7-PFUnA', 'D3-NMeFOSAA', 'NEtFOSAA', 'D5-NEtFOSAA', 'PFDS', 'PFDoA', '13C2-PFDoA', 'NMeFOSE', 'D7-NMeFOSE', 'NEtFOSE', '11Cl-PF3OUdS', 'D9-NEtFOSE', 'PFTrDA', 'PFTeDA', '13C2-PFTeDA']
Total compounds: 56
Without isotope prefix: 32
Found 56 S/N ratio columns for compounds:
  - PFBA_S/N
  - 13C4-PFBA_S/N
  - PFPeA_S/N
  - 13C5-PFPeA_S/N
  - HFPO-DA_S/N
  ... and 51 more

PFAS CALIBRATION S/N RATIO CHECK RESULTS
Total compounds analyzed: 56
Total calibration samples: 7

Compounds with ALL S/N ratios ≥ 

## 2 Calibration line:  Average Response Factor, recovery and RSE for EIS compounds 
To analyze all instrument linearity metrics, we first need to calculate the EIS concentrations found in the samples. 
We do this by caclulating the concentrations based on the formula on page 44 of EPA method 1633. 
To achieve this, we first need to calculate the average response factor per compound (EPA method 1633 page 28). 
Afterwards, we check if the recovery of the compound falls within the accepted limits denoted by EPA method 1633 (Table 6). 
We also already compute the concentrations for the QC samples, these are analyzed further in section 3.2 

We compute the RSE based on  the formula for RSE based on EPA method 1633 (section 10.3.3.3, page 28, option 2). The functions are loaded in a different script. 
If the RSE of a single compound remains above 20%, this means something is wrong with the linearity of the LC-MS TQ and the calculated concentrations in section 3 are not trustworthy. 
### 2.1 EIS compounds
MAKE SURE YOU CHANGE THE MATRIX TYPE FOR ACCURATE RECOVERIES

If you notice here certain calibration levels are consistently bad, there's an option to exclude them for further analysis in the 2nd code block. 
If you don't want to exclude any levels type *excluded_cal_level = 'None'*

The 2nd code block prints detailed information on the RSE. You can view detailed results by removing *%%capture* in the code block, or alternatively just observe the printed summary. 



In [78]:

from expected_concs import expected_concs_EIS, expected_concs_target_analytes

from analogs_compounds import target_EIS_analogs, EIS_NIS_analogs
from solutions import calibration_solutions

# computation of EIS concentration and recovery rate
from solutions import NIS_stock, EIS_stock
from allowed_recoveries import EIS_NIS_compound_recovery_aqueous

matrix_type = "solid"  # or "solid

# Select appropriate recovery limits based on matrix type
if matrix_type.lower() == "aqueous":
    EIS_NIS_compound_recovery = EIS_NIS_compound_recovery_aqueous
    OPR_recovery = OPR_recovery_aqueous
    IPR_recovery_rsd = IPR_recovery_rsd_aqueous
elif matrix_type.lower() == "solid":
    EIS_NIS_compound_recovery = EIS_NIS_compound_recovery_solid
    OPR_recovery = OPR_recovery_solid
    IPR_recovery_rsd = IPR_recovery_rsd_solid
else:
    raise ValueError(f"Unknown matrix type: {matrix_type}. Use 'aqueous' or 'solid'")

#calculations
eis_RFS = calculate_average_RFs_EIS(rawdata, EIS_NIS_analogs, calibration_solutions)


Ws_cal = 0.25#ml
dilution_NIS_stock_cal = 40 #dilution cal
dilution_EIS_stock_cal = 40 #dilution cal
spiked_amount_cal = 0.05 #ml
Df_cal = 1 #dilution factor
spiked_amount = 0.05 #ml

dilution_EIS_stock = 2 
spiked_amount_EIS =0.05 #ml

#calibration line
conc_EIS_cal, conc_QC = calculate_conc_EIS_cal(rawdata, eis_RFS, dilution_NIS_stock_cal, Ws_cal,
                          EIS_NIS_analogs, Df_cal, NIS_stock, spiked_amount_cal)

calculated_recoveries_EIS_cal = calculate_recoveries(conc_EIS_cal, expected_concs_EIS)

validation_results_cal= validate_recoveries(calculated_recoveries_EIS_cal, EIS_NIS_compound_recovery)
summary_cal = summarize_validation_results(validation_results_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries_cal = get_failed_recoveries(validation_results_cal)


AVERAGE RESPONSE FACTORS FOR EIS COMPOUNDS (n=24)


,Compound_Name,NIS_Analog,Mass_NIS,Mass_EIS,Average_Response_Factor,RSD_%
0,13C2-PFTeDA,13C2-PFDA,1.25,1.25,3.8839,5.97
1,13C5-PFPeA,13C2-PFHxA,2.50,5.00,0.6706,2.64
2,D5-NEtFOSA,13C4-PFOS,2.50,2.50,1.2215,10.50
3,13C4-PFHpA,13C2-PFHxA,2.50,2.50,0.9401,4.19
4,13C9-PFNA,13C5-PFNA,1.25,1.25,0.9708,2.45
5,13C7-PFUnA,13C2-PFDA,1.25,1.25,1.6687,7.47
6,D3-NMeFOSAA,13C4-PFOS,2.50,5.00,0.2049,4.97
7,13C3-PFHxS,18O2-PFHxS,2.50,2.50,1.3022,2.16
8,13C4-PFBA,13C3-PFBA,5.00,10.00,1.1220,0.32
9,13C8-PFOSA,13C4-PFOS,2.50,2.50,3.9500,4.82


Calibration Data:


,Name,Type,Level,13C2-PFTeDA_Conc_Calc,13C5-PFPeA_Conc_Calc,D5-NEtFOSA_Conc_Calc,13C4-PFHpA_Conc_Calc,13C9-PFNA_Conc_Calc,13C7-PFUnA_Conc_Calc,D3-NMeFOSAA_Conc_Calc,13C3-PFHxS_Conc_Calc,13C4-PFBA_Conc_Calc,13C8-PFOSA_Conc_Calc,13C2-6:2FTS_Conc_Calc,13C2-8:2FTS_Conc_Calc,13C3-HFPO-DA_Conc_Calc,D9-NEtFOSE_Conc_Calc,13C2-PFDoA_Conc_Calc,13C5-PFHxA_Conc_Calc,13C2-4:2FTS_Conc_Calc,13C8-PFOS_Conc_Calc,D3-NMeFOSA_Conc_Calc,13C8-PFOA_Conc_Calc,D5-NEtFOSAA_Conc_Calc,13C3-PFBS_Conc_Calc,D7-NMeFOSE_Conc_Calc,13C6-PFDA_Conc_Calc
0,CS1,Cal,L1,1.126445,4.912019,2.378428,2.444687,1.183689,1.194792,5.125671,2.588120,10.012923,2.480380,5.135017,4.710076,9.905255,25.340473,1.173739,2.484035,4.936306,2.663938,2.378188,2.539149,4.984211,2.556475,24.989409,1.139074
1,CS2,Cal,L2,1.192970,5.048464,3.058330,2.409584,1.280387,1.215018,4.759639,2.446245,10.049465,2.660696,4.721254,4.477186,10.125552,31.612544,1.153753,2.513343,5.504246,2.600694,3.174787,2.404669,4.627632,2.539901,31.330097,1.232340
2,CS3,Cal,L3,1.205650,4.764017,2.175399,2.581108,1.260816,1.437871,4.984139,2.522462,9.949198,2.310127,5.971254,7.233840,9.981981,22.439352,1.390928,2.493567,5.063694,2.479167,2.216502,2.407309,5.032895,2.479252,21.875110,1.355756
3,CS4,Cal,L4,1.350511,4.995899,2.380270,2.508510,1.241759,1.337943,4.864568,2.432806,10.007130,2.469810,4.991289,4.462928,10.116252,24.328267,1.370278,2.523351,4.583333,2.466270,2.337438,2.560582,5.060526,2.551687,24.233892,1.323391
4,CS5,Cal,L5,1.258497,4.980987,2.401965,2.389905,1.265709,1.215467,4.697413,2.458340,9.958111,2.483608,4.551394,4.966730,10.106371,24.670275,1.202360,2.505719,4.851380,2.408482,2.358251,2.547186,5.344737,2.498404,24.176081,1.205415
5,CS6,Cal,L6,1.279255,5.074187,2.654318,2.454792,1.276782,1.197264,5.494143,2.555483,10.019162,2.677025,4.577526,4.710076,10.041851,25.957316,1.257789,2.515488,5.111465,2.635169,2.630357,2.571259,5.147368,2.514855,26.442630,1.232408
6,CS7,Cal,L7,1.336672,5.223307,2.450880,2.711414,1.240987,1.151495,5.068326,2.496544,10.002228,2.418354,5.074042,4.420152,9.723901,20.641409,1.201152,2.464020,4.941614,2.246528,2.404916,2.469787,4.794737,2.359181,21.941748,1.261415


QC Data:


,Name,Type,Level,13C2-PFTeDA_Conc_Calc,13C5-PFPeA_Conc_Calc,D5-NEtFOSA_Conc_Calc,13C4-PFHpA_Conc_Calc,13C9-PFNA_Conc_Calc,13C7-PFUnA_Conc_Calc,D3-NMeFOSAA_Conc_Calc,13C3-PFHxS_Conc_Calc,13C4-PFBA_Conc_Calc,13C8-PFOSA_Conc_Calc,13C2-6:2FTS_Conc_Calc,13C2-8:2FTS_Conc_Calc,13C3-HFPO-DA_Conc_Calc,D9-NEtFOSE_Conc_Calc,13C2-PFDoA_Conc_Calc,13C5-PFHxA_Conc_Calc,13C2-4:2FTS_Conc_Calc,13C8-PFOS_Conc_Calc,D3-NMeFOSA_Conc_Calc,13C8-PFOA_Conc_Calc,D5-NEtFOSAA_Conc_Calc,13C3-PFBS_Conc_Calc,D7-NMeFOSE_Conc_Calc,13C6-PFDA_Conc_Calc
8,CS5,QC,L5,1.378994,5.322472,2.263201,2.286725,1.143902,1.323261,4.625427,2.488481,9.926471,2.414367,5.400697,4.420152,9.212392,24.924382,1.396302,2.531214,5.331741,2.418899,2.247174,2.604620,4.463158,2.529465,24.270521,1.355622
14,CS5,QC,L5,1.496177,5.411945,2.338723,2.342570,1.217166,1.325808,4.835286,2.485793,9.979501,2.516203,4.690767,4.429658,9.428621,24.554353,1.371667,2.541698,5.034501,2.484375,2.334589,2.383569,4.702632,2.580415,24.148720,1.375900


,Name,Type,D7-NMeFOSE_Recovery_%,D9-NEtFOSE_Recovery_%,13C4-PFBA_Recovery_%,13C3-HFPO-DA_Recovery_%,13C5-PFPeA_Recovery_%,13C2-4:2FTS_Recovery_%,13C2-6:2FTS_Recovery_%,13C2-8:2FTS_Recovery_%,D3-NMeFOSAA_Recovery_%,D5-NEtFOSAA_Recovery_%,13C5-PFHxA_Recovery_%,13C4-PFHpA_Recovery_%,13C8-PFOA_Recovery_%,13C3-PFBS_Recovery_%,13C3-PFHxS_Recovery_%,13C8-PFOS_Recovery_%,13C8-PFOSA_Recovery_%,D3-NMeFOSA_Recovery_%,D5-NEtFOSA_Recovery_%,13C9-PFNA_Recovery_%,13C6-PFDA_Recovery_%,13C7-PFUnA_Recovery_%,13C2-PFDoA_Recovery_%,13C2-PFTeDA_Recovery_%
0,CS1,Cal,99.957635,101.361892,100.129234,99.052546,98.240382,98.726115,102.700348,94.201521,102.513421,99.684211,99.361418,97.787469,101.565971,102.258999,103.524804,106.557540,99.215190,95.127509,95.137126,94.695097,91.125913,95.583388,93.899140,90.115605
1,CS2,Cal,125.320388,126.450177,100.494652,101.255522,100.969281,110.084926,94.425087,89.543726,95.192777,92.552632,100.533740,96.383363,96.186759,101.596032,97.849793,104.027778,106.427848,126.991499,122.333197,102.430985,98.587237,97.201414,92.300261,95.437576
2,CS3,Cal,87.500441,89.757408,99.491979,99.819809,95.280346,101.273885,119.425087,144.676806,99.682772,100.657895,99.742661,103.244336,96.292356,99.170063,100.898479,99.166667,92.405063,88.660065,87.015964,100.865266,108.460464,115.029664,111.274273,96.452020
3,CS4,Cal,96.935569,97.313066,100.071301,101.162520,99.917984,91.666667,99.825784,89.258555,97.291362,101.210526,100.934045,100.340389,102.423266,102.067475,97.312241,98.650794,98.792405,93.497502,95.210806,99.340750,105.871294,107.035417,109.622259,108.040887
4,CS5,Cal,96.704325,98.681099,99.581105,101.063706,99.619744,97.027601,91.027875,99.334601,93.948267,106.894737,100.228746,95.596213,101.887457,99.936159,98.333589,96.339286,99.344304,94.330032,96.078592,101.256696,96.433176,97.237370,96.188774,100.679729
5,CS6,Cal,105.770521,103.829265,100.191622,100.418507,101.483746,102.229299,91.550523,94.201521,109.882870,102.947368,100.619520,98.191682,102.850349,100.594215,102.219321,105.406746,107.081013,105.214267,106.172738,102.142563,98.592609,95.781147,100.623128,102.340431
6,CS7,Cal,87.766990,82.565638,100.022282,97.239014,104.466150,98.832272,101.480836,88.403042,101.366520,95.894737,98.560808,108.456547,98.791496,94.367235,99.861772,89.861111,96.734177,96.196652,98.035203,99.278945,100.913193,92.119614,96.092165,106.933752


Validation Summary:
Total_Measurements: 168
Passed: 168
Failed: 0
No_Data: 0
No_Reference: 0
Pass_Rate_%: 100.0


,Sample_Name,Sample_Type,Compound,Calculated_Recovery_%,Expected_Lower_%,Expected_Upper_%,Status,Compound_Type


In [79]:
%%capture
# RSE EIS Calibration line
excluded_cal_level = 'L3'
rse_results_cal_EIS = calculate_eis_rse_modified(conc_EIS_cal, expected_concs_EIS, p=2, exclude_levels = excluded_cal_level)


In [80]:
df_rse_eis=df_rse_results(rse_results_cal_EIS)
display(df_rse_eis)

,Compound,RSE_Percent,Calibration_Points,Relationship_Type,Parameters,Pass_Fail,Sum_Relative_Squared_Error
0,13C2-PFTeDA,7.700477,6,linear,2,PASS,0.005930
1,13C5-PFPeA,2.565886,6,linear,2,PASS,0.000658
2,D5-NEtFOSA,12.274936,6,linear,2,PASS,0.015067
3,13C4-PFHpA,5.297824,6,linear,2,PASS,0.002807
4,13C9-PFNA,3.208429,6,linear,2,PASS,0.001029
5,13C7-PFUnA,6.410284,6,linear,2,PASS,0.004109
6,D3-NMeFOSAA,6.575054,6,linear,2,PASS,0.004323
7,13C3-PFHxS,2.828122,6,linear,2,PASS,0.000800
8,13C4-PFBA,0.346105,6,linear,2,PASS,0.000012
9,13C8-PFOSA,5.114385,6,linear,2,PASS,0.002616


## 2.2 Target analytes 

### 2.2.1 Response Ratio (RR) and Response Factor (RF) for target ID compounds
Here we compute the response ratio for the ID compounds by using the formula from EPA 1633 on page 27 under section 10.3.3.2. For now we also fix some naming errors in the raw data for the target analytes. The only difference with the above calculation is that EIS analogs instead of direct pairs are used for the target EIS compounds. For ease, we call both RF.

- *Note 1*: NMeFoSA and NEtFoSa are calculated now but are not included in the PFAC30PAR ampoule we currently use in the lab, so these compounds cannot be in the calibration line. Use these results with care.
- *Note 2*:EPA 1633 mentions that PFTrDA recovery should improve by taking the average of the analog EIS compounds 13c2-PFTeDA and 13c2-PFDoa. Currently it only takes the EIS compound 13C2-PFTeDA as an analog.
- *Note 3*: EPA 1633 is capable of analyzing 40 compounds, our native standard solution only contains 30, so you will get errors that data for some compounds is not found. This is correct. If new ampoules are ordered for the other 10 compounds, everything needed for the calculation is there.
- *Note 4*: the FTS compounds only have 6 cal levels instead of 7.


In [81]:
from analogs_compounds import target_EIS_analogs

# Make a copy of rawdata to avoid modifying the original
rawdata_copy = rawdata.copy()
columns_to_rename = {}
for col in rawdata_copy.columns:
    new_col = col
    # Fix PFPHpA -> PFHpA
    if 'PFPHpA' in col:
        new_col = new_col.replace('PFPHpA', 'PFHpA')
    # Remove spaces from FTS compounds (e.g., "4:2 FTS" -> "4:2FTS")
    if ' FTS' in col:
        new_col = new_col.replace(' FTS', 'FTS')
    
    if new_col != col:
        columns_to_rename[col] = new_col

if columns_to_rename:
    rawdata_copy.rename(columns=columns_to_rename, inplace=True)

RF_target_analytes= calculate_average_RRs_targets(rawdata_copy, expected_concs_EIS, calibration_solutions, target_EIS_analogs)

Found 40 total target-IS pairs:
  NMeFOSE -> D7-NMeFOSE
  NEtFOSE -> D9-NEtFOSE
  PFBA -> 13C4-PFBA
  HFPO-DA -> 13C3-HFPO-DA
  PFPeA -> 13C5-PFPeA
  4:2FTS -> 13C2-4:2FTS
  6:2FTS -> 13C2-6:2FTS
  8:2FTS -> 13C2-8:2FTS
  NMeFOSAA -> D3-NMeFOSAA
  NEtFOSAA -> D5-NEtFOSAA
  PFHxA -> 13C5-PFHxA
  PFHpA -> 13C4-PFHpA
  PFOA -> 13C8-PFOA
  PFBS -> 13C3-PFBS
  PFHxS -> 13C3-PFHxS
  PFOS -> 13C8-PFOS
  PFOSA -> 13C8-PFOSA
  NMeFOSA -> D3-NMeFOSA
  NEtFOSA -> D5-NEtFOSA
  PFNA -> 13C9-PFNA
  PFDA -> 13C6-PFDA
  PFUnA -> 13C7-PFUnA
  PFDoA -> 13C2-PFDoA
  PFTeDA -> 13C2-PFTeDA
  PFTrDA -> 13C2-PFTeDA
  PFPeS -> 13C3-PFHxS
  PFHpS -> 13C8-PFOS
  PFNS -> 13C8-PFOS
  PFDS -> 13C8-PFOS
  PFDoS -> 13C8-PFOS
  ADONA -> 13C3-HFPO-DA
  PFMPA -> 13C5-PFPeA
  PFMBA -> 13C5-PFPeA
  NFDHA -> 13C5-PFHxA
  9Cl-PF3ONS -> 13C3-HFPO-DA
  11Cl-PF3OUdS -> 13C3-HFPO-DA
  PFEESA -> 13C5-PFHxA
  3:3FTCA -> 13C5-PFPeA
  5:3FTCA -> 13C5-PFHxA
  7:3FTCA -> 13C5-PFHxA


AVERAGE RESPONSE RATIOS FOR TARGET COMPOUNDS WITH

,Target_Compound,EIS_Isotope,Mass_Target (L4),Mass_EIS,Average_Response_Ratio,RSD_%,N_Levels,Levels_Used
0,NMeFOSE,D7-NMeFOSE,25.0,25.00,0.3891,12.93,7,"L1, L2, L3, L4, L5, L6, L7"
1,NEtFOSE,D9-NEtFOSE,25.0,25.00,0.3957,10.90,7,"L1, L2, L3, L4, L5, L6, L7"
2,PFBA,13C4-PFBA,10.0,10.00,0.3847,10.65,7,"L1, L2, L3, L4, L5, L6, L7"
3,HFPO-DA,13C3-HFPO-DA,10.0,10.00,0.4186,9.85,7,"L1, L2, L3, L4, L5, L6, L7"
4,PFPeA,13C5-PFPeA,5.0,5.00,0.4135,10.42,7,"L1, L2, L3, L4, L5, L6, L7"
5,4:2FTS,13C2-4:2FTS,10.0,5.00,1.5731,16.41,6,"L1, L2, L3, L4, L5, L6"
6,6:2FTS,13C2-6:2FTS,10.0,5.00,1.1561,7.99,6,"L1, L2, L3, L4, L5, L6"
7,8:2FTS,13C2-8:2FTS,10.0,5.00,1.0214,15.85,6,"L1, L2, L3, L4, L5, L6"
8,NMeFOSAA,D3-NMeFOSAA,2.5,5.00,0.4234,9.61,7,"L1, L2, L3, L4, L5, L6, L7"
9,NEtFOSAA,D5-NEtFOSAA,2.5,5.00,0.3852,14.64,7,"L1, L2, L3, L4, L5, L6, L7"


### 2.2.2 Recovery IPR and OPR for target analytes in calibration line 
Here we compare calculated recoveries for the target analytes in all calibration samples and compare them to the expected recoveries and maximum RSD for IPR (Table 5) 
* Note: OPR/LLOPR recovery is a little less strict. If needed for comparison exchange IPR_recovery_rsd_aqueous for OPR_recovery_aqueous. IPR considers mean recovery and RSD, OPR is per sample. Thus, you should not run the function mean_recoveries. 

In [83]:
from allowed_recoveries import IPR_recovery_rsd_aqueous, OPR_recovery_aqueous
sampletype = "Cal"
conc_target_analytes_cal = calculate_conc_targets(rawdata_copy,
                           RF_target_analytes,
                           EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal,
                           Df_cal, sampletype, debug_compound=None )
calculated_recoveries_target_cal = calculate_recoveries(conc_target_analytes_cal, expected_concs_target_analytes)

validation_results_target_cal= validate_recoveries(calculated_recoveries_target_cal, IPR_recovery_rsd)
summary_cal = summarize_validation_results(validation_results_target_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")

#for IPR, it is not necessarily failed if the below dataframe contains samples that do not pass. Rather, look at the output from mean_recoveries. 

failed_recoveries_cal = get_failed_recoveries(validation_results_target_cal)
#Only for IPR
IPR_target_cal = mean_recoveries(calculated_recoveries_target_cal, IPR_recovery_rsd_aqueous)
display(IPR_target_cal)

,Name,Type,Level,NMeFOSE_Conc_Calc,NEtFOSE_Conc_Calc,PFBA_Conc_Calc,HFPO-DA_Conc_Calc,PFPeA_Conc_Calc,4:2FTS_Conc_Calc,6:2FTS_Conc_Calc,8:2FTS_Conc_Calc,NMeFOSAA_Conc_Calc,NEtFOSAA_Conc_Calc,PFHxA_Conc_Calc,PFHpA_Conc_Calc,PFOA_Conc_Calc,PFBS_Conc_Calc,PFHxS_Conc_Calc,PFOS_Conc_Calc,PFOSA_Conc_Calc,NMeFOSA_Conc_Calc,NEtFOSA_Conc_Calc,PFNA_Conc_Calc,PFDA_Conc_Calc,PFUnA_Conc_Calc,PFDoA_Conc_Calc,PFTeDA_Conc_Calc,PFTrDA_Conc_Calc,PFPeS_Conc_Calc,PFHpS_Conc_Calc,PFNS_Conc_Calc,PFDS_Conc_Calc,ADONA_Conc_Calc,9Cl-PF3ONS_Conc_Calc,11Cl-PF3OUdS_Conc_Calc
0,CS1,Cal,L1,1.453913,1.529490,0.617952,0.657452,0.313739,0.537112,0.745670,1.041152,0.200155,0.134195,0.134001,0.156781,0.235717,0.152476,0.140678,0.148983,0.163293,0.153325,0.147565,0.170970,0.161696,0.165682,0.154649,0.159868,0.141950,0.149844,0.162993,0.131903,0.154282,0.625470,0.587704,0.558121
1,CS2,Cal,L2,4.853122,4.963669,1.957264,1.934272,0.967188,2.068239,1.817918,1.988692,0.440961,0.550678,0.087775,0.508462,0.443931,0.494387,0.488696,0.520950,0.479914,0.496371,0.531134,0.479130,0.481059,0.453895,0.507575,0.497375,0.499443,0.537692,0.479881,0.431975,0.456381,1.936171,1.881174,1.810321
2,CS3,Cal,L3,12.952316,12.772475,5.180441,5.050081,2.582214,5.222122,5.136469,4.589971,1.384975,1.366344,0.121380,1.273356,1.212335,1.272581,1.358439,1.229875,1.271479,1.297609,1.417223,1.248450,1.192717,1.254084,1.339253,1.249012,1.375808,1.290179,1.309165,1.385261,1.260239,5.484106,5.690916,5.450125
3,CS4,Cal,L4,27.550883,27.473207,10.820645,10.975088,5.449340,11.456988,11.337807,10.765384,2.646127,2.404215,3.893057,2.675383,2.543016,2.682422,2.855650,2.818615,2.713406,2.774325,1.775942,2.684662,2.818794,2.791811,2.636208,2.757020,2.756676,2.798896,2.715491,3.078923,2.771532,11.179187,11.205728,11.360074
4,CS5,Cal,L5,58.426483,56.588428,22.683315,23.078988,11.356521,23.226570,21.142261,18.092549,5.680357,5.690550,8.216750,5.716860,5.428323,5.680589,5.754231,5.652728,5.727083,5.462248,5.844769,5.695700,5.860797,5.815764,5.779292,5.836592,5.911523,5.539160,5.823845,5.870991,5.877954,22.430130,22.719990,22.458356
5,CS6,Cal,L6,121.280107,123.381991,48.713673,47.937328,24.357083,47.158883,47.045297,40.244216,11.398334,13.079778,17.400506,12.079069,11.511217,12.534360,11.581348,11.720166,12.043134,12.309348,13.610250,11.961357,12.247241,12.238854,11.898211,12.340422,12.316053,11.852496,11.729689,12.477627,12.233986,47.657573,48.164640,48.826827
6,CS7,Cal,L7,640.775836,627.374881,255.390609,247.724632,127.013580,240.076813,205.748203,209.473315,56.438145,61.788285,91.473759,62.504816,58.761121,63.740596,63.280922,64.583242,63.236991,63.479792,68.670543,63.689279,62.194069,62.568796,61.165695,59.280946,57.568856,60.478710,61.816068,60.271890,65.279554,240.221312,241.680914,267.985755


,Name,Type,PFBA_Recovery_%,4:2FTS_Recovery_%,6:2FTS_Recovery_%,8:2FTS_Recovery_%,HFPO-DA_Recovery_%,ADONA_Recovery_%,11Cl-PF3OUdS_Recovery_%,9Cl-PF3ONS_Recovery_%,PFPeA_Recovery_%,NMeFOSE_Recovery_%,NEtFOSE_Recovery_%,PFHxA_Recovery_%,PFHpA_Recovery_%,PFOA_Recovery_%,PFNA_Recovery_%,PFDA_Recovery_%,PFUnA_Recovery_%,PFDoA_Recovery_%,PFTrDA_Recovery_%,PFTeDA_Recovery_%,PFBS_Recovery_%,PFPeS_Recovery_%,PFHxS_Recovery_%,PFHpS_Recovery_%,PFOS_Recovery_%,PFOSA_Recovery_%,NMeFOSA_Recovery_%,NEtFOSA_Recovery_%,NMeFOSAA_Recovery_%,NEtFOSAA_Recovery_%,PFNS_Recovery_%,PFDS_Recovery_%
0,CS1,Cal,77.243988,67.138963,93.208737,130.144022,82.181483,78.183731,69.765096,73.462985,78.434673,72.695646,76.474497,67.000637,78.390705,117.858468,85.485014,80.848167,82.840807,77.324620,70.974838,79.934029,76.237979,74.921906,70.338906,81.496730,74.491362,81.646487,76.662652,73.782345,100.077660,67.097260,65.951323,77.140987
1,CS2,Cal,97.863197,103.411929,90.895896,99.434600,96.713624,96.808536,90.516069,94.058718,96.718841,97.062439,99.273390,17.555046,101.692394,88.786265,95.826030,96.211792,90.778967,101.515001,99.888593,99.475072,98.877433,107.538382,97.739298,95.976274,104.189918,95.982832,99.274134,106.226745,88.192299,110.135624,86.394951,91.276282
2,CS3,Cal,103.608827,104.442439,102.729376,91.799412,101.001617,109.682120,109.002500,113.818314,103.288559,103.618526,102.179802,9.710409,101.868500,96.986800,99.876016,95.417327,100.326730,107.140262,110.064629,99.920996,101.806497,103.214343,108.675092,104.733222,98.390037,101.718325,103.808710,113.377843,110.797985,109.307537,110.820893,100.819115
3,CS4,Cal,108.206447,114.569884,113.378072,107.653835,109.750877,111.791869,113.600743,112.057285,108.986796,110.203530,109.892829,155.722265,107.015320,101.720649,107.386499,112.751756,111.672441,105.448324,110.267060,110.280800,107.296885,111.955828,114.225985,108.619640,112.744586,108.536238,110.973018,71.037694,105.845099,96.168603,123.156917,110.861296
4,CS5,Cal,113.416577,116.132851,105.711303,90.462746,115.394940,112.150649,112.291781,113.599951,113.565210,116.852967,113.176855,164.334995,114.337203,108.566452,113.914007,117.215933,116.315285,115.585847,118.230461,116.731838,113.611775,110.783204,115.084621,116.476906,113.054565,114.541655,109.244951,116.895377,113.607146,113.811007,117.419817,117.559072
5,CS6,Cal,97.427347,94.317766,94.090593,80.488432,95.874656,95.315145,97.653653,96.329280,97.428333,97.024085,98.705593,139.204048,96.632556,92.089738,95.690859,97.977925,97.910834,95.185691,98.528424,98.723379,100.274879,94.819964,92.650786,93.837508,93.761329,96.345068,98.474784,108.881998,91.186675,104.638224,99.821014,97.871891
6,CS7,Cal,102.156244,96.030725,82.299281,83.789326,99.089853,96.088525,107.194302,96.672366,101.610864,102.524134,100.379981,146.358014,100.007706,94.017793,101.902847,99.510511,100.110073,97.865113,92.110170,94.849514,101.984953,96.765935,101.249476,98.905709,103.333187,101.179186,101.567667,109.872868,90.301033,98.861256,96.435025,104.447287


Validation Summary:
Total_Measurements: 224
Passed: 215
Failed: 9
No_Data: 0
No_Reference: 0
Pass_Rate_%: 95.98214285714286


,Sample_Name,Sample_Type,Compound,Calculated_Recovery_%,Expected_Lower_%,Expected_Upper_%,Status,Compound_Type
1,CS1,Cal,4:2FTS,67.138963,70,135,Fail,target
11,CS1,Cal,PFHxA,67.000637,70,135,Fail,target
30,CS1,Cal,PFNS,65.951323,70,140,Fail,target
43,CS2,Cal,PFHxA,17.555046,70,135,Fail,target
75,CS3,Cal,PFHxA,9.710409,70,135,Fail,target
107,CS4,Cal,PFHxA,155.722265,70,135,Fail,target
139,CS5,Cal,PFHxA,164.334995,70,135,Fail,target
171,CS6,Cal,PFHxA,139.204048,70,135,Fail,target
203,CS7,Cal,PFHxA,146.358014,70,135,Fail,target


,Compound,Compound_Type,Mean_Recovery_%,Expected_Lower_%,Expected_Upper_%,Recovery_Status,Observed_RSD_%,Expected_RSD_%,RSD_Status
0,PFBA,target,99.988947,70,135,Pass,11.505206,21,Pass
1,4:2FTS,target,99.434937,70,135,Pass,16.573856,27,Pass
2,6:2FTS,target,97.473323,70,135,Pass,10.691226,32,Pass
3,8:2FTS,target,97.681768,70,140,Pass,17.391603,33,Pass
4,HFPO-DA,target,100.001007,70,135,Pass,10.639705,23,Pass
5,ADONA,target,100.002939,70,135,Pass,12.273001,23,Pass
6,11Cl-PF3OUdS,target,100.003449,50,150,Pass,15.696967,35,Pass
7,9Cl-PF3ONS,target,99.999843,70,145,Pass,14.636404,30,Pass
8,PFPeA,target,100.004754,70,135,Pass,11.258754,23,Pass
9,NMeFOSE,target,99.997333,70,135,Pass,13.968267,29,Pass


### 2.2.3 RSE results target analytes
We make two results, one with all calibration levels (7 for general compounds, 6 for FTS) - rse_results_cal_target, and one with the calibration level as excluded earlier for EIS compounds - rse_results_excluded_cal.

In [72]:
%%capture
rse_results_cal_target, rse_results_excluded_cal = calculate_target_rse(conc_target_analytes_cal, expected_concs_target_analytes, p=2, level_to_exclude = excluded_cal_level )

In [84]:
df_rse_target=df_rse_results(rse_results_cal_target)
display(df_rse_target)

,Compound,RSE_Percent,Calibration_Points,Relationship_Type,Parameters,Pass_Fail,Sum_Relative_Squared_Error
0,NMeFOSE,15.301063,7,linear,2,PASS,0.023412
1,NEtFOSE,12.899971,7,linear,2,PASS,0.016641
2,PFBA,12.601935,7,linear,2,PASS,0.015881
3,HFPO-DA,11.655330,7,linear,2,PASS,0.013585
4,PFPeA,12.333935,7,linear,2,PASS,0.015213
5,4:2FTS,23.209642,6,quadratic,3,FAIL,0.053869
6,6:2FTS,9.784788,6,linear,2,PASS,0.009574
7,8:2FTS,19.406655,6,linear,2,PASS,0.037662
8,NMeFOSAA,11.376191,7,linear,2,PASS,0.012942
9,NEtFOSAA,17.321575,7,linear,2,PASS,0.030004


In [85]:
df_rse_target_excluded = df_rse_results(rse_results_excluded_cal) 
display(df_rse_target_excluded)

,Compound,RSE_Percent,Calibration_Points,Relationship_Type,Parameters,Pass_Fail,Sum_Relative_Squared_Error
0,NMeFOSE,17.011164,6,linear,2,PASS,0.028938
1,NEtFOSE,14.381366,6,linear,2,PASS,0.020682
2,PFBA,13.973370,6,linear,2,PASS,0.019526
3,HFPO-DA,13.021429,6,linear,2,PASS,0.016956
4,PFPeA,13.691376,6,linear,2,PASS,0.018745
5,4:2FTS,28.251790,5,quadratic,3,FAIL,0.079816
6,6:2FTS,11.188072,5,linear,2,PASS,0.012517
7,8:2FTS,26.825580,5,quadratic,3,FAIL,0.071961
8,NMeFOSAA,11.516207,6,linear,2,PASS,0.013262
9,NEtFOSAA,18.798634,6,linear,2,PASS,0.035339


## 3 Samples 

### 3.1 Recovery EIS in Samples 
Here we compute the recovery of the EIS compounds within the samples analyzed. 

In [86]:
# computation of EIS concentration and recovery rate
from solutions import NIS_stock, EIS_stock
from allowed_recoveries import EIS_NIS_compound_recovery_aqueous
DF_sample = 1 #dilution factor 
mass_bottle_filled = 499.8 #grams
mass_bottle_empty = 53.68 #grams
Ws = (mass_bottle_filled - mass_bottle_empty)/1000 #L
Ws_cal = 0.25#ml

# Create weights dictionary in grams
weights_dict = {
    'Sand500': 5.0,
    'Sand504': 5.04, 
    'Sand506': 5.06, 
    'Clay537': 5.37
}

rawdata_copy['Ws'] = rawdata_copy['Name'].map(weights_dict)

dilution_NIS_stock = 2 #1:1 dilution
dilution_NIS_stock_cal = 40 #dilution cal
dilution_EIS_stock_cal = 40 #dilution cal
spiked_amount_cal = 0.05 #ml
Df_cal = 1 #dilution factor
spiked_amount = 0.05 #ml

dilution_EIS_stock = 1 #normally 2 
spiked_amount_EIS =0.05 #ml

calculated_conc_EIS = calculate_conc_EIS(rawdata_copy, eis_RFS, dilution_NIS_stock,EIS_NIS_analogs, DF_sample, NIS_stock, spiked_amount)

#for run on 12-9-2025
expected_concs_EIS_doubled = {key: value * 2 for key, value in expected_concs_EIS.items()}
calculated_recoveries_EIS = calculate_recoveries(calculated_conc_EIS, expected_concs_EIS_doubled)

validation_results_samples= validate_recoveries(calculated_recoveries_EIS, EIS_NIS_compound_recovery_solid)
summary_samples = summarize_validation_results(validation_results_samples)
print("Validation Summary:")
for key, value in summary_samples['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries_samples = get_failed_recoveries(validation_results_samples)


  - Blank
  - Blank
  - Blank
  - Rinse

Calculated concentrations for 4 samples with Ws values


,Name,Type,Ws,13C2-PFTeDA_Conc_Calc,13C5-PFPeA_Conc_Calc,D5-NEtFOSA_Conc_Calc,13C4-PFHpA_Conc_Calc,13C9-PFNA_Conc_Calc,13C7-PFUnA_Conc_Calc,D3-NMeFOSAA_Conc_Calc,13C3-PFHxS_Conc_Calc,13C4-PFBA_Conc_Calc,13C8-PFOSA_Conc_Calc,13C2-6:2FTS_Conc_Calc,13C2-8:2FTS_Conc_Calc,13C3-HFPO-DA_Conc_Calc,D9-NEtFOSE_Conc_Calc,13C2-PFDoA_Conc_Calc,13C5-PFHxA_Conc_Calc,13C2-4:2FTS_Conc_Calc,13C8-PFOS_Conc_Calc,D3-NMeFOSA_Conc_Calc,13C8-PFOA_Conc_Calc,D5-NEtFOSAA_Conc_Calc,13C3-PFBS_Conc_Calc,D7-NMeFOSE_Conc_Calc,13C6-PFDA_Conc_Calc
10,Sand500,Sample,5.00,2.309438,10.124515,3.523946,4.340230,2.253039,2.929601,9.400927,4.856013,18.406863,4.135000,9.098432,10.337452,17.563357,40.275603,2.778476,4.852745,10.684713,4.541667,3.648234,4.517185,9.289474,4.652311,38.531774,2.568892
11,Sand504,Sample,5.04,2.557809,10.949165,3.960132,4.561160,2.616711,2.722274,9.413467,4.912131,20.074254,4.839386,10.486664,9.260517,18.669523,46.648537,2.551142,5.124606,11.447697,4.806292,4.253936,4.599577,9.209221,5.313530,45.360855,2.442522
12,Sand506,Sample,5.06,2.136493,9.636450,3.590156,3.900119,2.261821,2.306401,8.479264,4.436282,17.634201,4.198979,9.072317,8.369152,16.700891,41.042497,2.318628,4.551746,9.752922,4.294694,3.596964,4.762397,8.399209,4.758374,38.314280,2.305011
13,Clay537,Sample,5.37,1.898685,8.549419,3.279998,3.503134,2.037738,2.033483,7.711442,3.856280,15.524950,3.871779,7.254946,7.390375,14.608897,38.240379,2.014543,3.994353,8.535071,4.035454,3.475403,3.868593,7.572528,4.006198,37.300241,1.998016


,Name,Type,D7-NMeFOSE_Recovery_%,D9-NEtFOSE_Recovery_%,13C4-PFBA_Recovery_%,13C3-HFPO-DA_Recovery_%,13C5-PFPeA_Recovery_%,13C2-4:2FTS_Recovery_%,13C2-6:2FTS_Recovery_%,13C2-8:2FTS_Recovery_%,D3-NMeFOSAA_Recovery_%,D5-NEtFOSAA_Recovery_%,13C5-PFHxA_Recovery_%,13C4-PFHpA_Recovery_%,13C8-PFOA_Recovery_%,13C3-PFBS_Recovery_%,13C3-PFHxS_Recovery_%,13C8-PFOS_Recovery_%,13C8-PFOSA_Recovery_%,D3-NMeFOSA_Recovery_%,D5-NEtFOSA_Recovery_%,13C9-PFNA_Recovery_%,13C6-PFDA_Recovery_%,13C7-PFUnA_Recovery_%,13C2-PFDoA_Recovery_%,13C2-PFTeDA_Recovery_%
10,Sand500,Sample,77.063548,80.551205,92.034314,87.816787,101.245154,106.847134,90.984321,103.374525,94.009273,92.894737,97.054899,86.804595,90.343700,93.046211,97.120258,90.833333,82.700000,72.964683,70.478919,90.121549,102.755694,117.184035,111.139020,92.377507
11,Sand504,Sample,90.721711,93.297075,100.371272,93.347616,109.491648,114.476966,104.866642,92.605166,94.134673,92.092210,102.492110,91.223198,91.991532,106.270593,98.242626,96.125835,96.787724,85.078725,79.202646,104.668421,97.700881,108.890963,102.045677,102.312375
12,Sand506,Sample,76.628560,82.084994,88.171003,83.504457,96.364503,97.529225,90.723169,83.691519,84.792640,83.992095,91.034916,78.002382,95.247939,95.167473,88.725647,85.893877,83.979587,71.939287,71.803119,90.472834,92.200428,92.256025,92.745140,85.459717
13,Clay537,Sample,74.600482,76.480759,77.624752,73.044487,85.494188,85.350714,72.549459,73.903746,77.114421,75.725277,79.887061,70.062676,77.371852,80.123962,77.125604,80.709076,77.435589,69.508056,65.599962,81.509500,79.920639,81.339314,80.581730,75.947390


Validation Summary:
Total_Measurements: 96
Passed: 96
Failed: 0
No_Data: 0
No_Reference: 0
Pass_Rate_%: 100.0


,Sample_Name,Sample_Type,Compound,Calculated_Recovery_%,Expected_Lower_%,Expected_Upper_%,Status,Compound_Type


## 3.2 QC samples
In this section we will check the results of the QC samples and assessing if they meet recovery limits. We have already computed EIS recovery for QC earlier. Here, we also compute the concentration for the target analytes in the QC samples.

In [87]:
sample_QC = "QC"
QC_level = 5
conc_target_analytes_QC = calculate_conc_targets(rawdata_copy,
                           RF_target_analytes,
                           EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal,
                           Df_cal, sample_QC, debug_compound=None )

calculated_recoveries_EIS_QC = calculate_recoveries(conc_QC, expected_concs_EIS)
expected_concs_level =  filter_expected_concs_by_level(expected_concs_target_analytes,QC_level)
calculated_recoveries_target_QC = calculate_recoveries(conc_target_analytes_QC, expected_concs_level)

validation_results_target_QC= validate_recoveries(calculated_recoveries_target_QC, OPR_recovery_solid)

summary_cal = summarize_validation_results(validation_results_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries_cal = get_failed_recoveries(validation_results_cal)


,Name,Type,Level,NMeFOSE_Conc_Calc,NEtFOSE_Conc_Calc,PFBA_Conc_Calc,HFPO-DA_Conc_Calc,PFPeA_Conc_Calc,4:2FTS_Conc_Calc,6:2FTS_Conc_Calc,8:2FTS_Conc_Calc,NMeFOSAA_Conc_Calc,NEtFOSAA_Conc_Calc,PFHxA_Conc_Calc,PFHpA_Conc_Calc,PFOA_Conc_Calc,PFBS_Conc_Calc,PFHxS_Conc_Calc,PFOS_Conc_Calc,PFOSA_Conc_Calc,NMeFOSA_Conc_Calc,NEtFOSA_Conc_Calc,PFNA_Conc_Calc,PFDA_Conc_Calc,PFUnA_Conc_Calc,PFDoA_Conc_Calc,PFTeDA_Conc_Calc,PFTrDA_Conc_Calc,PFPeS_Conc_Calc,PFHpS_Conc_Calc,PFNS_Conc_Calc,PFDS_Conc_Calc,ADONA_Conc_Calc,9Cl-PF3ONS_Conc_Calc,11Cl-PF3OUdS_Conc_Calc
8,CS5,QC,L5,54.687975,53.470337,21.563783,21.709135,10.582106,21.788248,18.341683,21.350332,5.283322,6.284913,7.605208,5.265164,4.993234,5.569304,5.135447,5.375998,5.166960,5.283034,2.291383,5.672180,5.103512,5.153285,5.129184,5.288012,4.835871,5.274335,5.358489,5.674223,5.670870,21.645106,22.391844,22.408065
14,CS5,QC,L5,46.065009,45.402504,18.199276,18.239404,8.947994,19.125710,16.521590,16.581770,4.765716,4.675459,6.565250,4.330131,4.184255,4.490551,4.563985,4.417384,4.512129,4.604407,5.156425,4.409993,4.470702,4.496504,4.487247,4.493929,4.211958,4.279146,4.305523,4.603596,4.615174,17.956128,18.642316,18.985648


,Name,Type,D7-NMeFOSE_Recovery_%,D9-NEtFOSE_Recovery_%,13C4-PFBA_Recovery_%,13C3-HFPO-DA_Recovery_%,13C5-PFPeA_Recovery_%,13C2-4:2FTS_Recovery_%,13C2-6:2FTS_Recovery_%,13C2-8:2FTS_Recovery_%,D3-NMeFOSAA_Recovery_%,D5-NEtFOSAA_Recovery_%,13C5-PFHxA_Recovery_%,13C4-PFHpA_Recovery_%,13C8-PFOA_Recovery_%,13C3-PFBS_Recovery_%,13C3-PFHxS_Recovery_%,13C8-PFOS_Recovery_%,13C8-PFOSA_Recovery_%,D3-NMeFOSA_Recovery_%,D5-NEtFOSA_Recovery_%,13C9-PFNA_Recovery_%,13C6-PFDA_Recovery_%,13C7-PFUnA_Recovery_%,13C2-PFDoA_Recovery_%,13C2-PFTeDA_Recovery_%
8,CS5,QC,97.082083,99.697528,99.264706,92.123925,106.449448,106.634820,108.013937,88.403042,92.508541,89.263158,101.248570,91.468993,104.184788,101.178608,99.539241,96.755952,96.574684,89.886951,90.528039,91.512155,108.449721,105.860850,111.704183,110.319524
14,CS5,QC,96.594881,98.217411,99.795009,94.286213,108.238891,100.690021,93.815331,88.593156,96.705710,94.052632,101.667937,93.702798,95.342761,103.216618,99.431731,99.375000,100.648101,93.383577,93.548915,97.373300,110.071981,106.064601,109.733359,119.694122


,Name,Type,PFBA_Recovery_%,4:2FTS_Recovery_%,6:2FTS_Recovery_%,8:2FTS_Recovery_%,HFPO-DA_Recovery_%,ADONA_Recovery_%,11Cl-PF3OUdS_Recovery_%,9Cl-PF3ONS_Recovery_%,PFPeA_Recovery_%,NMeFOSE_Recovery_%,NEtFOSE_Recovery_%,PFHxA_Recovery_%,PFHpA_Recovery_%,PFOA_Recovery_%,PFNA_Recovery_%,PFDA_Recovery_%,PFUnA_Recovery_%,PFDoA_Recovery_%,PFTrDA_Recovery_%,PFTeDA_Recovery_%,PFBS_Recovery_%,PFPeS_Recovery_%,PFHxS_Recovery_%,PFHpS_Recovery_%,PFOS_Recovery_%,PFOSA_Recovery_%,NMeFOSA_Recovery_%,NEtFOSA_Recovery_%,NMeFOSAA_Recovery_%,NEtFOSAA_Recovery_%,PFNS_Recovery_%,PFDS_Recovery_%
8,CS5,QC,107.818917,108.941240,91.708413,106.751659,108.545673,108.225528,112.040324,111.959220,105.821065,109.375950,106.940675,152.104167,105.303278,99.864672,113.44360,102.070248,103.065694,102.583677,96.717413,105.760237,111.386089,105.486710,102.708934,107.169782,107.519969,103.339195,105.660688,45.827656,105.666436,125.698267,113.484455,113.417397
14,CS5,QC,90.996380,95.628552,82.607948,82.908849,91.197021,89.780641,94.928239,93.211582,89.479939,92.130018,90.805007,131.305005,86.602623,83.685092,88.19986,89.414048,89.930086,89.744935,84.239160,89.878575,89.811024,85.582923,91.279709,86.110466,88.347677,90.242574,92.088141,103.128497,95.314320,93.509182,92.071911,92.303478


Validation Summary:
Total_Measurements: 168
Passed: 168
Failed: 0
No_Data: 0
No_Reference: 0
Pass_Rate_%: 100.0


,Sample_Name,Sample_Type,Compound,Calculated_Recovery_%,Expected_Lower_%,Expected_Upper_%,Status,Compound_Type
